### Librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
import glob

import funciones as fn #funciones es el nombre del script dnd hemos creado las funciones

### Carga de datos

In [2]:
path_dataset1 = '..\\datosTransformados\\'
lista = ['accesos','actividades', 'conocimientos', 'cursos', 'cursos_en_programas', 'itinerarios', 'notas_cursos', 'plantas', 'programas', 'roles', 'tutores_perfil', 'usuarios']

# Creamos un diccionario en blanco con los nombres de la lista como keys
dfs = {}
for i in lista:
    dfs[i] = ''

# get data file names
filenames = glob.glob(path_dataset1 + "*.csv")

# cargar dfs en el diccionario 
for i, valuei in enumerate(lista):
    for j, filename in enumerate(filenames):
        if i==j:
            dfs[valuei] = pd.read_csv(filename, index_col= [0])
    
# separar los keys del diccionario en df diferentes (con su nombre)
for i in dfs.keys():
    globals()['%s' % i.replace(" ", "_").lower()] = dfs[i]

### CSV cursos comenzados y no-comenzados recuento

In [22]:
notas_cursos['comenzado'] = None
for i in range(len(notas_cursos['online_progress'])):
    if notas_cursos.loc[i,'online_progress'] == 0:
        notas_cursos.loc[i,'comenzado'] = 0

    else:
        notas_cursos.loc[i,'comenzado'] = 1

In [23]:
cursos_comenzados = pd.DataFrame(notas_cursos['comenzado'].value_counts().reset_index(), columns=['comenzado', 'recuento'])
cursos_comenzados = cursos_comenzados.transpose()
cursos_comenzados.drop(index='recuento', axis= 0, inplace=True)
cursos_comenzados = cursos_comenzados.rename(columns={0: 'no_comenzado', 1:'comenzado'}).reset_index(drop= True)
cursos_comenzados

,no_comenzado,comenzado
0,3321.0,577.0


In [24]:
file_name2 = 'cursos_comenzados.csv'
path_dataset2 = '..\\datosTransformados\\Datos_powerBI\\'

In [25]:
# Eliminar los .csv existentes (antiguos) en la carpeta y descargar la última version
fn.eliminar_csv(path_dataset2) #eliminar
fn.convertir_csv(df=cursos_comenzados, filename=file_name2,path_dataset = path_dataset2 ) #descargar


### CSV Horas usuarios

In [27]:
user_curso = cursos.merge(notas_cursos , on= ['course_name', 'course_id'] , how = "right")
#horas_alumnos = user_curso.groupby('user_id')['hours'].sum().reset_index()
user_cursos = user_curso[user_curso['hours'] != 0]
fn.convertir_csv(df = user_cursos , filename = 'user_cursos.csv', path_dataset = path_dataset2 )

### CSV Cursos mas relevantes

In [33]:
#Preparamos los datos: los cursos que mas aparecen en notas_cursos, es decir, los mas realizados y por tanto populares
cursos_popu = notas_cursos['course_name'].value_counts().to_frame().reset_index().head(10)
cursos_popu.rename(columns= {'index': 'course_name', 'course_name': 'num_usuarios'}, inplace=True)

fn.convertir_csv(df = cursos_popu , filename = 'cursos_popu.csv', path_dataset = path_dataset2)